In [1]:
from pathlib import Path
from typing import Union
from utils import resource_utils
from plugins.resources import patch_resources
import pandas as pd

INFO     Directory PATH=C:\Users\pj8\PycharmProjects\elAPI_Plugins\tests\nul\elapi from                 path.py:156
         SOURCE=nul\elapi could not be found. An attempt to create directory                                       
         C:\Users\pj8\PycharmProjects\elAPI_Plugins\tests\nul\elapi will be made.                                  

ERROR    [WinError 3] Das System kann den angegebenen Pfad nicht finden:                                path.py:176
         'C:\\Users\\pj8\\PycharmProjects\\elAPI_Plugins\\tests\\nul\\elapi'                                       

In [2]:
session = resource_utils.FixedCategoryEndpoint()

In [3]:
session = resource_utils.FixedCategoryEndpoint()
categories = session.get().json()
df_categories = pd.json_normalize(categories)
print("ID  Title")
for category in categories:
    print(f"{category['id']}: {category['title']}")

ID  Title
434: Information
489: Recipes
488: ISA - Study
487: ISA - Investigation
483: BZH info
482: Instruments and Machines
456: labfolder project
437: Group meeting
435: other
420: Group landing page
421: Antibody
422: mutant
423: Resource overview
424: Methods
426: Plasmid
427: compound
428: meta-analysis


In [4]:
cat = session.get(426).json()

In [5]:
res_session = resource_utils.FixedResourceEndpoint()
res = res_session.get(query = {"cat" : 426})

In [6]:
df_res  = pd.json_normalize(res.json())
type(df_res)

pandas.core.frame.DataFrame

In [7]:
df_res.drop(columns=["team", "elabid", "category", "locked", "lockedby", "locked_at", "userid", "canread", "canwrite", "available","lastchangeby","state","events_start","content_type","created_at","access_key","is_bookable","canbook","book_max_minutes","book_max_slots","book_can_overlap","book_is_cancellable","book_cancel_minutes","status","custom_id","timestamped","timestampedby","timestamped_at","book_users_can_in_past","is_procurable","proc_pack_qty","proc_price_notax","proc_price_tax","proc_currency","page","type","status_color","category_color","recent_comment","has_comment","tags_id","events_start_itemid","next_step","firstname","lastname","orcid","up_item_id"], inplace = True)

#TODO has_attachment -> Yes/No

In [8]:
df_extra_fields = df_res[["id","metadata"]]
df_extra_fields.reset_index()
df_extra_fields

,id,metadata
0,10130,"{""extra_fields"": {""MM section"": {""type"": ""text..."
1,9694,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
2,9693,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
3,9692,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
4,9691,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
5,9690,"{""extra_fields"": {""link"": {""type"": ""items"", ""v..."
6,9595,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
7,9594,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
8,9593,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
9,9592,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."


In [9]:
import json
parsed_metadata = df_extra_fields["metadata"].apply(json.loads)
extra_fields = parsed_metadata.apply(lambda d: d.get("extra_fields", {}))

In [10]:
df_extra_fields

,id,metadata
0,10130,"{""extra_fields"": {""MM section"": {""type"": ""text..."
1,9694,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
2,9693,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
3,9692,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
4,9691,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
5,9690,"{""extra_fields"": {""link"": {""type"": ""items"", ""v..."
6,9595,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
7,9594,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
8,9593,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."
9,9592,"{""extra_fields"": {""insert"": {""type"": ""text"", ""..."


In [11]:
flattened_values = extra_fields.apply(
    lambda field_dict: {
        k: v.get("value") if isinstance(v, dict) else None
        for k, v in field_dict.items()
    }
)

In [12]:
df_flat = pd.DataFrame(flattened_values.tolist())

In [13]:
df_flat["id"] = df_extra_fields["id"].values  # or use index if needed

In [14]:
df_final = df_res.drop(columns="metadata").merge(df_flat, on="id", how="left")

In [16]:
df_final.to_excel("res2.xlsx", index = False)